クロスデバイス　Crawlingとの併用

In [ ]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

9199	楽天株式会社

In [ ]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

60003706	クローリング_サッカー関連

In [ ]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 9199
seg_id = 60003706
seg_name = 'crawling'

#クロスデバイステーブル
xd_table = 'segment.drawbridge_device'

In [10]:
seed_table = f"dm_tmp.ya_seed_advid_{advertiser_id}_{seg_id}_{seg_name}_{today_yymmdd}"

query = (
    f"""
    SELECT smn_uid 
    FROM  segment.crawling_seg_user_partitioned_by_ymd
    WHERE id = 102932 and smn_uid>0
    GROUP BY smn_uid ORDER BY max(concat_ws('-',year,month,day)) desc,sum(cnt) desc
    limit 5000000
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_seed = ir.sql_to_pandas(query)
        ir.upload_dataframe(seed_table, df_seed, ['bigint'])

df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

In [11]:
print(seed_table)

dm_tmp.ya_seed_advid_9199_60003706_crawling_190530


In [12]:
df_seed.shape[0]

682522

In [13]:
#XD拡張

uid_table = f"dm_tmp.ya_xd_advid_{advertiser_id}_{seg_id}_{seg_name}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    ,xd2.device
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in (
    'windows_computer'
    ,'linux_computer'
    ,'mac'
    ,'ipad'
    ,'android_tablet'
    ,'windows_tablet') 
    and xd2.smn_uid not in (select smn_uid from {seed_table})
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    ir.upload_dataframe(uid_table, df_fix, ['bigint','string']) 
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [14]:
print(uid_table)

dm_tmp.ya_xd_advid_9199_60003706_crawling_190530


In [15]:
df_fix.shape[0]

1612366

In [16]:
df_fix.groupby(["device"])["smn_uid"].count()

device
android_tablet       148452
ipad                  54321
linux_computer         4837
mac                   43987
windows_computer    1359865
windows_tablet          904
Name: smn_uid, dtype: int64